In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import hppfcl
import numpy as np
import time

import jax
from jax import jacfwd, jacrev, vmap, jit
from jax import numpy as jnp
jax.config.update('jax_platform_name', 'cpu')


In [11]:
a = jnp.zeros((2,3))
a+1

Array([[1., 1., 1.],
       [1., 1., 1.]], dtype=float32)

In [5]:
def func(a, b, c):
    return a + b*c

%timeit func(1, 2, 3)
jfunc = jax.jit(func)
jfunc(1, 2, 3)
%timeit jfunc(3, 4, 5)

65 ns ± 0.798 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)
3.1 µs ± 28.3 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [10]:
a = np.random.randn(2,2)
b = np.random.randn(2)
print(a)
print(b)
np.matmul(a,b)

[[ 2.43756467 -0.0879469 ]
 [ 0.58766458  0.55374537]]
[ 0.23796872 -0.32896611]


array([ 0.6089957 , -0.04231767])

In [11]:
-0.64130257*-0.78487848+-0.48791498*0.0685164

0.46991440842602167

In [3]:
from iLQR.cost.collision_checker.collision_checker import CollisionChecker
from iLQR.config import Config
config = Config()
c = CollisionChecker(config)
c.ego_object.computeVolume()

0.17599999999999996

In [36]:
def test_jnp(state):
    psi = state[3]
    pose = jnp.array([[jnp.cos(psi), -jnp.sin(psi), 0, state[0]],
                        [jnp.sin(psi), jnp.cos(psi), 0, state[1]],
                        [0,0,1,0],
                        [0,0,0,1]])
def test_np(state):
    psi = state[3]
    pose = np.array([[np.cos(psi), -np.sin(psi), 0, state[0]],
                        [np.sin(psi), np.cos(psi), 0, state[1]],
                        [0,0,1,0],
                        [0,0,0,1]])

def test_np2(state):
    psi = state[3]
    pose = np.eye(4)
    pose[:2,:2] = np.array([[np.cos(psi), -np.sin(psi)],
                        [np.sin(psi), np.cos(psi)]])
    pose[:2,3] = state[:2]    
    # pose = np.array([[np.cos(psi), -np.sin(psi), 0, state[0]],
    #                     [np.sin(psi), np.cos(psi), 0, state[1]],
    #                     [0,0,1,0],
    #                     [0,0,0,1]])
    print(np.linalg.inv(pose))

In [43]:
state = np.array([10,223,332141,42,5])
# test_jit = jax.jit(test_jnp)
# test_jit(a)
# %timeit test_jnp(a)
# %timeit test_jit(a)
test_np2(state)


psi = state[3]
cp = np.cos(psi)
sp = np.sin(psi)


ego_rotm_inv = np.array([[cp, sp, 0],
                                [-sp, cp, 0],
                                [0,0,1]])
print(ego_rotm_inv)

ego_trans_inv = np.array([-(cp*state[0] + sp*state[1]),
                                (sp*state[0] - cp*state[1]), 0])
print(ego_trans_inv)

[[ -0.39998531  -0.91652155  -0.         208.38415834]
 [  0.91652155  -0.39998531   0.          80.03150976]
 [  0.           0.           1.           0.        ]
 [  0.           0.           0.           1.        ]]
[[-0.39998531 -0.91652155  0.        ]
 [ 0.91652155 -0.39998531  0.        ]
 [ 0.          0.          1.        ]]
[208.38415834  80.03150976   0.        ]


In [14]:
psi = 0.2
ego_pose = np.eye(3)
ego_pose[:2, :2] = np.array([[np.cos(psi), -np.sin(psi)],
                    [np.sin(psi), np.cos(psi)]])
ego_pose[:2, -1] = np.array([0.1, 0.2])

a = np.random.randn(20,2)
a1 = np.append(a, np.ones((a.shape[0], 1)), axis=1)
np.matmul(ego_pose, a1.T).T


array([[-4.87895609e-02,  1.28958298e-01,  1.00000000e+00],
       [-7.29240983e-01, -9.33807754e-01,  1.00000000e+00],
       [ 2.12250963e+00,  3.95486446e-01,  1.00000000e+00],
       [ 6.35460608e-02,  6.85852281e-01,  1.00000000e+00],
       [-3.14227080e-01,  1.08101259e+00,  1.00000000e+00],
       [ 3.92041032e-02,  1.86943719e-01,  1.00000000e+00],
       [ 1.44409341e+00, -3.17301589e-01,  1.00000000e+00],
       [-6.77330308e-01, -6.32213574e-01,  1.00000000e+00],
       [ 5.34211859e-01, -1.76807833e-01,  1.00000000e+00],
       [-1.91351006e+00, -1.05146760e-01,  1.00000000e+00],
       [ 1.62978843e+00, -3.62259751e-02,  1.00000000e+00],
       [ 2.31774189e+00, -3.84368253e-01,  1.00000000e+00],
       [-1.67053643e-01,  5.92782701e-01,  1.00000000e+00],
       [ 6.59105113e-01, -5.14296673e-04,  1.00000000e+00],
       [ 5.34064276e-01, -1.31819726e-01,  1.00000000e+00],
       [ 1.03911710e+00,  2.13842455e-01,  1.00000000e+00],
       [ 8.93899476e-01,  3.96651945e-01

In [3]:
verts = hppfcl.StdVec_Vec3f()
faces = hppfcl.StdVec_Triangle()
verts.extend(
    [
        np.array([0.5, 0.5, 0]),
        np.array([0.5, -0.5, 0]),
        np.array([-0.5, 0.5, 0]),
        np.array([-0.5, -0.5, 0]),
        np.array([0.5, 0.5, 0.5]),
        np.array([0.5, -0.5, 0.5]),
        np.array([-0.5, 0.5, 0.5]),
        np.array([-0.5, -0.5, 0.5]) ,
        # np.array([0, 0, -0.5]),
        # np.array([0, 0, 0.5]),
    ]
)
c = hppfcl.Convex.convexHull(verts, True, None)
c.points()
b = hppfcl.Box(1, 1, 1)

obj1 = hppfcl.CollisionObject(c, np.eye(3), np.array([0,0,0]))
obj2 = hppfcl.CollisionObject(b, np.eye(3), np.array([-2.0,2.0,0]))

request = hppfcl.DistanceRequest()
result = hppfcl.DistanceResult()
hppfcl.distance(obj2, obj1, request, result)
print(result.min_distance)




1.4142135623730951


In [4]:
result.getNearestPoint1()

array([-1.5       ,  1.5       ,  0.16666667])

In [ ]:
def dis_func(x, y):
    # collision object
    obj1 = hppfcl.CollisionObject(hppfcl.Box(1, 1, 1), np.eye(3), np.array([0,0,0]))
    obj2 = hppfcl.CollisionObject(hppfcl.Box(1, 1, 1), np.eye(3), np.array([x,y,0]))

    request = hppfcl.DistanceRequest()
    result = hppfcl.DistanceResult()
    hppfcl.distance(obj1, obj2, request, result)
    print(result.getNearestPoint1())
    print(result.getNearestPoint2())
    return result.min_distance

dis_func(-1,-1)

[-0.5 -0.5 -0.5]
[-0.5 -0.5 -0.5]


-1.7976931348623157e+308

In [ ]:
def test(a, b):
    return a**3+b**3+3*a*b

In [ ]:
import numpy as np

N = 1000
D = 2   
e = 1e-5
z = np.random.randn(N, D)


np.mean(test(z[:,0]*e+1, z[:,1]*e+1)[...,np.newaxis]*z, axis=0)





array([0.016788  , 0.05044002])

In [ ]:
jax.grad(test, argnums=(0,1))(1.0,1.0)

(Array(6., dtype=float32, weak_type=True),
 Array(6., dtype=float32, weak_type=True))